In [2]:
"""
isPublic 업데이트 통합 스크립트
파라미터만 수정해서 실행하세요!
"""
import json
import pymysql

# ═══════════════════════════════════════════════════════════
# 여기만 수정하세요! ✏️
# ═══════════════════════════════════════════════════════════

# 파일 경로
BASE_JSON = "three_form_insert.json"  #컨트롤러  # 전체 문서 JSON
PUBLIC_LIST_JSON = "one_isPublicCrawling_result.json" #컨트롤러  # 공개 문서 목록

# 매칭 방식 선택
MATCH_BY = "id"     # "id" 또는 "title"

# 출력 파일 (None이면 원본 덮어쓰기)
# OUTPUT_JSON = None  # 예: "approval_documents_updated.json"
OUTPUT_JSON = "three_ispublic_insert.json" #컨트롤러  # 예: "approval_documents_updated.json"

# DB 업데이트 여부
UPDATE_DB = True

# DB 설정
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': '2021010120251031', #컨트롤러
    'table': 'approval_documents' #컨트롤러 # 테이블명
}

# ═══════════════════════════════════════════════════════════
# 메인 로직 (수정 금지)
# ═══════════════════════════════════════════════════════════

def update_public_status(base_json, public_list_json, match_by="id", 
                        output_json=None, update_db=False, db_config=None):
    """
    공개 문서 목록과 전체 문서를 매칭하여 isPublic 필드 업데이트
    """
    
    print("\n" + "="*70)
    print("📚 JSON 파일 읽기")
    print("="*70)
    
    # 1. JSON 파일 읽기
    try:
        with open(base_json, 'r', encoding='utf-8') as f:
            all_docs = json.load(f)
        print(f"✓ 전체 문서: {len(all_docs)}건")
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {base_json}")
        return {'매칭_성공': 0, '매칭_실패': 0, 'DB_업데이트': False}
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 오류: {e}")
        return {'매칭_성공': 0, '매칭_실패': 0, 'DB_업데이트': False}
    
    try:
        with open(public_list_json, 'r', encoding='utf-8') as f:
            public_docs = json.load(f)
        print(f"✓ 공개 문서: {len(public_docs)}건")
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {public_list_json}")
        return {'매칭_성공': 0, '매칭_실패': 0, 'DB_업데이트': False}
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 오류: {e}")
        return {'매칭_성공': 0, '매칭_실패': 0, 'DB_업데이트': False}
    
    # 2. 공개 문서 세트 생성
    print("\n" + "="*70)
    print(f"🔍 매칭 준비 (방식: {match_by.upper()})")
    print("="*70)
    
    if match_by == "id":
        # 문서ID로 매칭
        public_set = {str(doc.get('문서ID', '')).strip() for doc in public_docs}
        print(f"✓ 공개 문서 ID 세트: {len(public_set)}개")
    elif match_by == "title":
        # 제목으로 매칭 (정규화)
        public_set = {doc.get('제목', '').strip() for doc in public_docs}
        print(f"✓ 공개 문서 제목 세트: {len(public_set)}개")
    else:
        print(f"❌ 잘못된 매칭 방식: {match_by}")
        return {'매칭_성공': 0, '매칭_실패': 0, 'DB_업데이트': False}
    
    # 3. 매칭 및 isPublic 업데이트
    print("\n" + "="*70)
    print("🔄 isPublic 업데이트 중...")
    print("="*70)
    
    matched_count = 0
    unmatched_count = 0
    matched_ids = []  # DB 업데이트용
    
    for doc in all_docs:
        # 기본값: isPublic = 0 (비공개)
        doc['isPublic'] = False
        
        # 매칭 키 추출
        if match_by == "id":
            key = str(doc.get('sourceId', '')).strip()
        else:  # title
            key = doc.get('title', '').strip()
        
        # 공개 문서 세트에 있으면 isPublic = 1
        if key and key in public_set:
            doc['isPublic'] = True
            matched_count += 1
            matched_ids.append(doc.get('sourceId'))
            
            # 처음 5개만 출력
            if matched_count <= 5:
                print(f"  ✓ 매칭: {key[:50]}...")
        else:
            unmatched_count += 1
    
    print(f"\n{'─'*70}")
    print(f"✅ 매칭 성공: {matched_count}건")
    print(f"⚪ 매칭 실패: {unmatched_count}건")
    print(f"{'─'*70}")
    
    # 4. JSON 파일 저장
    print("\n" + "="*70)
    print("💾 JSON 파일 저장")
    print("="*70)
    
    output_path = output_json if output_json else base_json
    
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(all_docs, f, ensure_ascii=False, indent=2)
        print(f"✓ 저장 완료: {output_path}")
    except Exception as e:
        print(f"❌ 저장 실패: {e}")
        return {'매칭_성공': matched_count, '매칭_실패': unmatched_count, 'DB_업데이트': False}
    
    # 5. DB 업데이트 (선택적)
    db_updated = False
    if update_db and matched_count > 0 and db_config:
        print("\n" + "="*70)
        print("💾 MariaDB 업데이트")
        print("="*70)
        
        db_updated = update_database(matched_ids, db_config)
    
    # 6. 결과 반환
    return {
        '매칭_성공': matched_count,
        '매칭_실패': unmatched_count,
        'DB_업데이트': db_updated
    }


def update_database(source_ids, db_config):
    """
    DB에서 해당 source_id들의 is_public을 1로 업데이트
    """
    
    conn = None
    cursor = None
    
    try:
        # DB 연결
        conn = pymysql.connect(
            host=db_config['host'],
            user=db_config['user'],
            password=db_config['password'],
            database=db_config['database'],
            charset='utf8mb4'
        )
        print(f"✓ DB 연결 성공: {db_config['host']}/{db_config['database']}")
        
        cursor = conn.cursor()
        
        table_name = db_config.get('table', 'approval_documents')
        
        # 테이블 존재 확인
        cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
        if not cursor.fetchone():
            print(f"⚠️ 테이블이 존재하지 않습니다: {table_name}")
            print(f"   approval_documents 테이블을 사용합니다.")
            table_name = "approval_documents"
        
        # 1단계: 모든 is_public을 0으로 초기화
        print(f"\n1️⃣ 모든 문서를 비공개(0)로 초기화...")
        reset_sql = f"UPDATE {table_name} SET is_public = 0"
        cursor.execute(reset_sql)
        reset_count = cursor.rowcount
        print(f"   ✓ {reset_count}건 초기화 완료")
        
        # 2단계: 매칭된 문서만 is_public = 1로 업데이트
        print(f"\n2️⃣ 공개 문서를 공개(1)로 업데이트...")
        
        if not source_ids:
            print("   ⚠️ 업데이트할 문서가 없습니다.")
            conn.commit()
            return True
        
        # IN 절을 사용한 일괄 업데이트
        placeholders = ','.join(['%s'] * len(source_ids))
        update_sql = f"UPDATE {table_name} SET is_public = 1 WHERE source_id IN ({placeholders})"
        
        cursor.execute(update_sql, source_ids)
        update_count = cursor.rowcount
        print(f"   ✓ {update_count}건 업데이트 완료")
        
        # 커밋
        conn.commit()
        print(f"\n✅ DB 업데이트 완료!")
        print(f"   - 초기화: {reset_count}건")
        print(f"   - 공개 설정: {update_count}건")
        
        return True
        
    except pymysql.Error as e:
        print(f"\n❌ DB 오류:")
        print(f"   Error Code: {e.args[0]}")
        print(f"   Error Message: {e.args[1]}")
        return False
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        return False
        
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
            print("✓ DB 연결 종료")


# ═══════════════════════════════════════════════════════════
# 실행 (아래 코드는 수정하지 마세요)
# ═══════════════════════════════════════════════════════════

if __name__ == "__main__":
    print(f"""
╔═══════════════════════════════════════════════════════════╗
║              isPublic 업데이트 실행 설정                  ║
╚═══════════════════════════════════════════════════════════╝
📂 전체 문서:    {BASE_JSON}
📂 공개 문서:    {PUBLIC_LIST_JSON}
🔍 매칭 방식:    {MATCH_BY.upper()}
💾 출력 파일:    {OUTPUT_JSON if OUTPUT_JSON else '원본 덮어쓰기'}
💾 DB 업데이트:  {'예' if UPDATE_DB else '아니오'}
💾 DB 서버:      {DB_CONFIG['host']}:{DB_CONFIG.get('port', 3306)}
💾 DB 이름:      {DB_CONFIG['database']}
💾 DB 테이블:    {DB_CONFIG['table']}
    """)
    
    confirm = input("계속 진행하시겠습니까? (y/N): ")
    if confirm.lower() != 'y':
        print("❌ 취소됨")
        exit(0)
    
    # 실행
    result = update_public_status(
        base_json=BASE_JSON,
        public_list_json=PUBLIC_LIST_JSON,
        match_by=MATCH_BY,
        output_json=OUTPUT_JSON,
        update_db=UPDATE_DB,
        db_config=DB_CONFIG
    )
    
    print(f"\n✨ 완료! {result['매칭_성공']}건 업데이트됨")


╔═══════════════════════════════════════════════════════════╗
║              isPublic 업데이트 실행 설정                  ║
╚═══════════════════════════════════════════════════════════╝
📂 전체 문서:    three_form_insert.json
📂 공개 문서:    one_isPublicCrawling_result.json
🔍 매칭 방식:    ID
💾 출력 파일:    three_ispublic_insert.json
💾 DB 업데이트:  예
💾 DB 서버:      localhost:3306
💾 DB 이름:      2021010120251031
💾 DB 테이블:    approval_documents
    


계속 진행하시겠습니까? (y/N):  y



📚 JSON 파일 읽기
✓ 전체 문서: 8175건
✓ 공개 문서: 13건

🔍 매칭 준비 (방식: ID)
✓ 공개 문서 ID 세트: 13개

🔄 isPublic 업데이트 중...
  ✓ 매칭: 15371818...
  ✓ 매칭: 16068037...
  ✓ 매칭: 17256577...
  ✓ 매칭: 17745880...
  ✓ 매칭: 17887082...

──────────────────────────────────────────────────────────────────────
✅ 매칭 성공: 13건
⚪ 매칭 실패: 8162건
──────────────────────────────────────────────────────────────────────

💾 JSON 파일 저장
✓ 저장 완료: three_ispublic_insert.json

💾 MariaDB 업데이트
✓ DB 연결 성공: localhost/2021010120251031

1️⃣ 모든 문서를 비공개(0)로 초기화...
   ✓ 13건 초기화 완료

2️⃣ 공개 문서를 공개(1)로 업데이트...
   ✓ 13건 업데이트 완료

✅ DB 업데이트 완료!
   - 초기화: 13건
   - 공개 설정: 13건
✓ DB 연결 종료

✨ 완료! 13건 업데이트됨
